In [12]:
import numpy as np                                      
# import matplotlib.pyplot as plt                         
# import matplotlib.patches as patches
# import seaborn as sns
# import scipy.signal as signal 
# from scipy.io import loadmat
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import loader_2015_epoch as loader2015
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import sklearn.ensemble as ensemble

In [13]:
X_train, y_train = loader2015.load(1, 3)
X_test, y_test = loader2015.load(40, 44)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6468, 32, 410), (6468,), (6660, 32, 410), (6660,))

In [14]:
# Define our filter variables
fs = 512                      # Hz; sampling rate
dt = 1000. / fs                 # ms; time between samples
sdt = dt#np.round(dt).astype(int); # rounded dt so that we can index samples
hp = 1                        # Hz; our low cut for our bandpass
lp = 24.                        # Hz; our high cut for our bandpass
num_taps = 31                   # Number of taps/coefficients of FIR filter

# Create our filter coefficients
# Note: by defining 'fs' we don't divide our windows by the Nyquist
# Note: for FIR filters, a is always 1
# b = signal.firwin(numtaps=num_taps, cutoff=[hp, lp], pass_zero='bandpass', fs=fs)
# a = 1

# Define ERP-related variables
epoch_start = 0    # ms
epoch_end = 800    # ms
baseline_start = 0 # ms
baseline_end = 100 # ms
erp_start = 200    # ms
erp_end = 800      # ms

# Let's translate these from time into index space to save time later
e_s = np.round(epoch_start / sdt).astype(int)     # epoch start
e_e = np.round(epoch_end / sdt).astype(int)       # epoch end
bl_s = np.round(baseline_start / sdt).astype(int) # baseline start
bl_e = np.round(baseline_end / sdt).astype(int)   # baseline end
erp_s = np.round(erp_start / sdt).astype(int)     # ERP component window start
erp_e = np.round(erp_end / sdt).astype(int)       # ERP component window end

In [15]:
# pre-processing, inplace
def preprocess(x):
    for i in range(x.shape[0]):
        # correct DC offset of signal
        x[i] = x[i] - np.mean(x[i], axis=1).reshape(-1, 1)
        
        sd_every_chan = np.std(x[i], axis=1).reshape(-1, 1)
        x[i] = x[i] / sd_every_chan
        
        # bandpass filter
        #x[i] = signal.filtfilt(b, a, x[i], axis=1)
        # baseline correction
        #x[i] = x[i] - np.mean(x[i][bl_s:bl_e], axis=0)

preprocess(X_train)
preprocess(X_test)

In [16]:
# downsample

num_points = 6; # we will divide our window into num_points means
# Define a simple windowed means function
def wm(x, start, end, num_points):
    num_trials = x.shape[0] # assumes first dem is numb observations
    num_chans = x.shape[1] # assumes last dim is num channels
    len_time = x.shape[2] # assumes second dim is time
    w = np.round((end-start)/num_points).astype(int)
    y = np.zeros((num_trials, num_chans, num_points))
    for i in range(0, num_points):
        s = start + (w * i)
        e = s + w
        if e > len_time:
            e = len_time
        y[:,:,i] = np.mean(x[:,:,s:e], axis=2)
    return y

X_train = wm(X_train, erp_s, erp_e, num_points)
X_test = wm(X_test, erp_s, erp_e, num_points)

In [17]:
# Since our X is 3D, we must flatten our data. We will then transpose it for sklearn
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Let's print out the new shape
print('X_train shape is now: ' + str(X_train.shape))
print('X_test  shape is now: ' + str(X_test.shape))

X_train shape is now: (6468, 192)
X_test  shape is now: (6660, 192)


In [107]:
#np.nan_to_num(X_train, copy=False)
from sklearn.model_selection import train_test_split

# split the data into training and testing
X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# clf = RandomForestClassifier(n_jobs=-1, class_weight='balanced')

# params = {
#     'n_estimators': [16, 64, 128, 256],
#     'max_features': ['sqrt', 'log2', None],
#     'max_depth' : [4, 20, 100, None]
# }
# gscv = GridSearchCV(clf, params, cv=5, n_jobs=-1, verbose=1)
# gscv.fit(X_tr, y_tr)

# print("best params: ", gscv.best_params_)
# print("best score: ", gscv.best_score_)

svc1 = SVC(kernel='linear', C=1, class_weight='balanced', probability = True, max_iter=1000)
svc2 = SVC(kernel='linear', C=1, class_weight='balanced', probability = True, max_iter=1000)
svc3 = SVC(kernel='poly', C=1, class_weight='balanced', probability = True, max_iter=1000)
rf1 = RandomForestClassifier(n_estimators=64, max_features=None, class_weight='balanced', n_jobs=-1)

clf = ensemble.VotingClassifier(estimators=[('svc1', svc1), 
                                            ('svc2', svc2),
                                            ('svc', svc3), 
                                            ('rf1', rf1)], voting='hard', n_jobs=-1,
                                )
clf.fit(X_tr, y_tr)

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


VotingClassifier(estimators=[('svc1',
                              SVC(C=1, class_weight='balanced', kernel='linear',
                                  max_iter=1000, probability=True)),
                             ('svc2',
                              SVC(C=1, class_weight='balanced', kernel='linear',
                                  max_iter=1000, probability=True)),
                             ('svc',
                              SVC(C=1, class_weight='balanced', kernel='poly',
                                  max_iter=1000, probability=True)),
                             ('rf1',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_features=None,
                                                     n_estimators=64,
                                                     n_jobs=-1))],
                 n_jobs=-1)

In [108]:
clf.score(X_te, y_te)

0.7179289026275116

In [109]:
y_train_pred = clf.predict(X_tr)
print(confusion_matrix(y_tr, y_train_pred))
print(np.unique(y_tr, return_counts=True))
print(np.unique(y_train_pred, return_counts=True))
print("tn, fp, fn, tp", confusion_matrix(y_tr, y_train_pred).ravel())
correct_count = 0
for gt, p in zip(y_tr, y_train_pred):
    if gt == p:
        correct_count += 1
print("accuracy for training: ", correct_count / len(y_tr))

[[3296 1015]
 [ 448  415]]
(array([-1.,  1.]), array([4311,  863]))
(array([-1.,  1.]), array([3744, 1430]))
tn, fp, fn, tp [3296 1015  448  415]
accuracy for training:  0.717240046385775


In [110]:
from sklearn.metrics import confusion_matrix
y_train_pred = clf.predict(X_te)
print(confusion_matrix(y_te, y_train_pred))
print(np.unique(y_te, return_counts=True))
print(np.unique(y_train_pred, return_counts=True))
print("tn, fp, fn, tp", confusion_matrix(y_te, y_train_pred).ravel())
correct_count = 0
for gt, p in zip(y_te, y_train_pred):
    if gt == p:
        correct_count += 1
print("accuracy for training_test: ", correct_count / len(y_te))

[[819 261]
 [104 110]]
(array([-1.,  1.]), array([1080,  214]))
(array([-1.,  1.]), array([923, 371]))
tn, fp, fn, tp [819 261 104 110]
accuracy for training_test:  0.7179289026275116


In [111]:
np.nan_to_num(X_test, copy=False)

y_pred = clf.predict(X_test)
print("tn, fp, fn, tp", confusion_matrix(y_test, y_pred).ravel())
print(clf.score(X_test, y_test))
print(np.unique(y_test, return_counts=True))
print(np.unique(y_pred, return_counts=True))
correct_count = 0
for gt, p in zip(y_test, y_pred):
    if gt == p:
        correct_count += 1
print("accuracy for testing: ", correct_count / len(y_test))

tn, fp, fn, tp [4141 1409  643  467]
0.6918918918918919
(array([-1.,  1.]), array([5550, 1110]))
(array([-1.,  1.]), array([4784, 1876]))
accuracy for testing:  0.6918918918918919
